In [ ]:
import numpy as np

In [ ]:
import pints
import numpy as np


class CustomLogPDF(pints.LogPDF):
    def __init__(self, contributing_x, delta):
        self.contributing_x = contributing_x
        self.delta = delta

    def __call__(self, x):
        if np.any(x < 0) or np.any(x > 1):
            return -np.inf  # log(0) = -inf for out-of-bounds values
        probabilities = np.prod(
            np.exp(-self.delta * (x - self.contributing_x) ** 2), axis=1
        )
        pdf_value = np.max(probabilities)
        if pdf_value == 0:
            return -np.inf  # log(0) = -inf
        return np.log(pdf_value)

    def n_parameters(self):
        return self.contributing_x.shape[1]


def pdf(x, contributing_x, delta):
    if np.any(x < 0) or np.any(x > 1):
        return 0  # return 0 if x is not in [0, 1]
    probabilities = np.prod(np.exp(-delta * (x - contributing_x) ** 2), axis=1)
    return np.max(probabilities)


contributing_x = np.array(
    [[0.1, 0.9], [0.9, 0.9], [0.5, 0.1], [0.49, 0], [0, 1], [0.2, 0.8]]
)


input_bounds = np.array([[0, 1], [0, 1]]).T


n_samples = 8


delta = 300


x0 = np.ones((1, 2)) * 0.5  # initial guess

boundaries = pints.RectangularBoundaries(input_bounds[0], input_bounds[1])
print(boundaries.lower())
logpdf = CustomLogPDF(contributing_x, delta)
print(contributing_x[5, :])
logpdf(np.array([0.13, 0.7]))
sampler = pints.MCMCController(logpdf, 1, x0=x0, method=pints.SliceDoublingMCMC)
sampler.set_log_to_screen(False)
sampler.set_max_iterations(100)
sampler
samples = sampler.run()
samples = samples.reshape(-1, 2)  # shape (100, 2)
phi = np.vectorize(lambda x, y: x * y / 2)
phi(*samples.T)

In [ ]:
contributing_x = np.array(
    [
        [0.1, 0.9],
        [0.2, 0.8],
        [0.5, 0],
        [0.5, 0.5],
        [0.6, 0.6],
        [1, 1],
    ]
)


def rejection_sampling(pdf, n_samples, input_dim, random_acceptance=0.02):
    samples = []
    while len(samples) < n_samples:
        sample = np.random.uniform(low=0, high=1, size=input_dim)
        if np.random.uniform(0, 0.6) < pdf(
            sample
        ):  # 0.5 as pdf hardly ever higher values, this might also do a little bit of exploration
            samples.append(sample)
        elif np.random.uniform(0, 1) < random_acceptance:  # some extra exploration
            samples.append(sample)
    return np.array(samples)

In [ ]:
from moon.possibilistic.distributions import Points
from moon.dust.plotting import plot


def ensure_2d(array):
    return np.atleast_2d(array).T if array.ndim == 1 else array


pi_x = lambda x1, x2: np.min(
    [
        np.max(4 * np.power(0.5 - np.abs(x1 - 0.5), 2), 0),
        np.max(4 * np.power(0.5 - np.abs(x2 - 0.5), 2), 0),
    ]
)  # Eq14
pi_x = np.vectorize(pi_x)
phi = lambda x1, x2: (x1 - 0.5) ** 2 + x2  # , x2)
phi = np.vectorize(phi)
x_input = Points.from_bounds(
    [[0, 1], [0, 1]], sampling="grid", resolution=np.ceil(np.sqrt(10000)).astype(int)
)
x_input.values = np.random.uniform(0, 1, size=(200, 2))
memberships = np.array(pi_x(*x_input.values.T))
x_input.memberships = memberships
# y_out = Points(values=np.array(phi(*x_input.values.T)))  # Eq13 (model)
# y_out.memberships = memberships
y = np.array(phi(*x_input.values.T)).T
y_memberships = memberships
print(x_input.values.shape)
print(y.shape)
print(y_memberships.shape)
hist, edges = np.histogramdd(y, bins=21)
print(edges)
selected_indices = []

for dim in range(1):
    for bin in range(21):
        bin_mask = np.logical_and(
            ensure_2d(y)[:, dim] >= edges[dim][bin],
            ensure_2d(y)[:, dim] <= edges[dim][bin + 1],
        )
        if not np.any(bin_mask):
            print("no bin mask")
            continue
        max_index = np.argmax(y_memberships[bin_mask])
        index_in_y = np.where(bin_mask)[0][max_index]
        selected_indices.append(index_in_y)
        # find the corresponding x value
        x_value = x_input.values[bin_mask][max_index]
        x_value = x_input.values[index_in_y]
        # index
print(selected_indices)
selected_indices = np.unique(selected_indices)
x_contributing = x_input.values[selected_indices]

plot(Points(x_input.values[selected_indices]))
plot(Points(y[selected_indices], y_memberships[selected_indices]))

new_pdf = lambda x: pdf(x, x_contributing, delta=300)
new_samples = rejection_sampling(new_pdf, n_samples=100, input_dim=2)
plot(Points(new_samples), s=1)

In [ ]:
vectorized_function = lambda x: x[:, 0] + x[:, 1] ** 2 - x[:, 2]


# Example usage
def ensure_2d(array):
    return np.atleast_2d(array).T if array.ndim == 1 else array


vectorized_input = np.random.uniform(0, 1, size=(5, 3))
# vectorized_input = np.random.uniform(0, 1, size=3)
# print(vectorized_function(ensure_2d(vectorized_input)))
vectorized_input[1, :]

In [ ]:
def ensure_2d(array):
    return np.atleast_2d(array).T if array.ndim == 1 else array


def rejection_sampling(pdf, n_samples, input_dim, delta=300, random_acceptance=0.04):
    samples = []
    N = 1000
    tryouts = np.random.uniform(low=0, high=1, size=(N, input_dim))
    # evaluate pdf for each tryout
    # other numpy vectorized optimzed way to compute probabilities
    probabilities = np.array([pdf(tryout) for tryout in tryouts])
    U = np.random.uniform(low=0, high=0.7, size=N)
    accepted = tryouts[U < probabilities]
    print(f"Accepted {len(accepted)} out of {N} samples")
    return accepted


def pdf(x, x_contributing, delta):
    if np.any(x < 0) or np.any(x > 1):
        return 0  # return 0 if x is not in [0, 1]
    probabilities = np.prod(
        np.exp(-delta * ensure_2d((x - x_contributing) ** 2)), axis=1
    )
    return np.max(probabilities)


from moon.possibilistic.distributions import Points

# 1d contributing x
contributing_x = np.array([[0.1], [0.5], [0.9]])
# samples = np.random.uniform(0, 1, size=(2000, 1))
# pdf_values = np.array([pdf(x, contributing_x, delta=300) for x in samples])
# # plot(Points(samples, pdf_values), s=1)
print(f"\n1d rejection sampling:")
samples = rejection_sampling(
    lambda x: pdf(x, contributing_x, delta=300), n_samples=100, input_dim=1
)
# 2d contributing x
print(f"\n2d rejection sampling:")
contributing_x = np.array([[0.1, 0.9], [0.5, 0.5], [0.9, 0.8]])
samples = rejection_sampling(
    lambda x: pdf(x, contributing_x, delta=200), n_samples=100, input_dim=2
)
samples = np.random.uniform(0, 1, size=(2000, 2))
pdf_values = np.array([pdf(x, contributing_x, delta=200) for x in samples])
plot(Points(samples, pdf_values), s=100)


# 3d contributing x
print(f"\n3d rejection sampling:")
contributing_x = np.array([[0.1, 0.9, 0.5], [0.5, 0.5, 0.5], [0.9, 0.8, 0.5]])
samples = rejection_sampling(lambda x: pdf(x, contributing_x, delta=50), 100, 3)
samples = np.random.uniform(0, 1, size=(2000, 3))
pdf_values = np.array([pdf(x, contributing_x, delta=100) for x in samples])
plot(Points(samples, pdf_values))

# 4d contributing x
print(f"\n4d rejection sampling:")
contributing_x = np.array(
    [[0.1, 0.9, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5], [0.9, 0.8, 0.5, 0.5]]
)
samples = rejection_sampling(lambda x: pdf(x, contributing_x, delta=100), 100, 4)

# 5d contributing x
print(f"\n5d rejection sampling:")
contributing_x = np.array(
    [[0.1, 0.9, 0.5, 0.9, 0.5], [0.5, 0.5, 0.3, 0.3, 0.5], [0.9, 0.8, 0, 0.5, 0]]
)
samples = rejection_sampling(lambda x: pdf(x, contributing_x, delta=100), 100, 5)
samples = np.random.uniform(0, 1, size=(2000, 5))
pdf_values = np.array([pdf(x, contributing_x, delta=100) for x in samples])
plot(Points(samples, pdf_values))